In [99]:
import pandas as pd
import numpy as np
from kerasbeats import prep_time_series, NBeatsModel
import tensorflow.python.keras.backend as K

In [100]:
df = pd.read_csv('DailyDelhiClimateTest.csv', parse_dates = ['date'], index_col = 'date')
df.sort_index(inplace = True)

In [101]:
# prep a univariate time series for N-Beats
X, y = prep_time_series(df['meantemp'], lookback = 7, horizon = 1)

In [104]:
df.head(10)

,meantemp,humidity,wind_speed,meanpressure
date,,,,
2017-01-01,15.913043,85.869565,2.743478,59.000000
2017-01-02,18.500000,77.222222,2.894444,1018.277778
2017-01-03,17.111111,81.888889,4.016667,1018.333333
2017-01-04,18.700000,70.050000,4.545000,1015.700000
2017-01-05,18.388889,74.944444,3.300000,1014.333333
2017-01-06,19.318182,79.318182,8.681818,1011.772727
2017-01-07,14.708333,95.833333,10.041667,1011.375000
2017-01-08,15.684211,83.526316,1.950000,1015.550000
2017-01-09,14.571429,80.809524,6.542857,1015.952381


In [102]:
X

array([[15.91304348, 18.5       , 17.11111111, 18.7       , 18.38888889,
        19.31818182, 14.70833333],
       [18.5       , 17.11111111, 18.7       , 18.38888889, 19.31818182,
        14.70833333, 15.68421053],
       [17.11111111, 18.7       , 18.38888889, 19.31818182, 14.70833333,
        15.68421053, 14.57142857],
       [18.7       , 18.38888889, 19.31818182, 14.70833333, 15.68421053,
        14.57142857, 12.11111111],
       [18.38888889, 19.31818182, 14.70833333, 15.68421053, 14.57142857,
        12.11111111, 11.        ],
       [19.31818182, 14.70833333, 15.68421053, 14.57142857, 12.11111111,
        11.        , 11.78947368],
       [14.70833333, 15.68421053, 14.57142857, 12.11111111, 11.        ,
        11.78947368, 13.23529412],
       [15.68421053, 14.57142857, 12.11111111, 11.        , 11.78947368,
        13.23529412, 13.2       ],
       [14.57142857, 12.11111111, 11.        , 11.78947368, 13.23529412,
        13.2       , 16.43478261],
       [12.11111111, 11.    

In [106]:
K.reverse(X, axes=0)

<tf.Tensor: shape=(107, 7), dtype=float64, numpy=
array([[32.55555556, 34.        , 33.5       , 34.5       , 34.25      ,
        32.9       , 32.875     ],
       [31.        , 32.55555556, 34.        , 33.5       , 34.5       ,
        34.25      , 32.9       ],
       [31.22222222, 31.        , 32.55555556, 34.        , 33.5       ,
        34.5       , 34.25      ],
       [30.5       , 31.22222222, 31.        , 32.55555556, 34.        ,
        33.5       , 34.5       ],
       [29.66666667, 30.5       , 31.22222222, 31.        , 32.55555556,
        34.        , 33.5       ],
       [29.25      , 29.66666667, 30.5       , 31.22222222, 31.        ,
        32.55555556, 34.        ],
       [29.25      , 29.25      , 29.66666667, 30.5       , 31.22222222,
        31.        , 32.55555556],
       [27.85714286, 29.25      , 29.25      , 29.66666667, 30.5       ,
        31.22222222, 31.        ],
       [27.125     , 27.85714286, 29.25      , 29.25      , 29.66666667,
        30.5 

In [107]:
X[:, -1:]

array([[14.70833333],
       [15.68421053],
       [14.57142857],
       [12.11111111],
       [11.        ],
       [11.78947368],
       [13.23529412],
       [13.2       ],
       [16.43478261],
       [14.65      ],
       [11.72222222],
       [13.04166667],
       [14.61904762],
       [15.26315789],
       [15.39130435],
       [18.44      ],
       [18.11764706],
       [18.34782609],
       [21.        ],
       [16.17857143],
       [16.5       ],
       [14.86363636],
       [15.66666667],
       [16.44444444],
       [16.125     ],
       [15.25      ],
       [17.09090909],
       [15.63636364],
       [18.7       ],
       [18.63157895],
       [16.88888889],
       [15.125     ],
       [15.7       ],
       [15.375     ],
       [14.66666667],
       [15.625     ],
       [16.25      ],
       [16.33333333],
       [16.875     ],
       [17.57142857],
       [20.25      ],
       [21.3       ],
       [21.125     ],
       [22.36363636],
       [23.375     ],
       [21

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
shuffle = False, test_size = 0.2)

In [30]:
nbeats = NBeatsModel(model_type = 'generic', lookback = 7, horizon = 1)
nbeats.fit(X, y, epochs = 30, verbose = False)

In [31]:
preds = pd.DataFrame(index = np.arange(len(y_train) + len(y_test)))
preds['Real Value'] = np.hstack([y_train[:, 0], y_test[:, 0]])
preds['Prediction'] = np.hstack([nbeats.predict(X_train)[:, 0], nbeats.predict(X_test)[:, 0]])

10/10 [==============================] - 0s 16ms/step


In [32]:
import plotly.express as px

fig = px.line(preds[['Real Value', 'Prediction']], title = 'Real vs Predicted Values for N-Beats')
fig.add_vline(x = 1164, line_width=3, line_dash="dash", line_color="green")
fig.add_annotation(x = 1000, y = 10,
 text="Training Data",
 font=dict(size=15),
 showarrow=False)
fig.add_annotation(x = 1300, y = 10,
 text="Test Data",
 font=dict(size=15),
 showarrow=False)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Real Value<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Real Value',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Real Value',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([   0,    1,    2, ..., 1452, 1453, 1454]),
              'xaxis': 'x',
              'y': array([ 8.85714286, 14.        , 11.        , ..., 14.0952381 , 15.05263158,
                          10.        ]),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Prediction<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Prediction',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Prediction',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([   0,    1,    2, ..., 1452, 1453, 1454]),
              'xaxis': 'x',
              'y': array([ 7.23235178,  9.04274368, 14.2092104 , ..., 15.52538395, 14.24836445,
                          15.10861397]),
              'yaxis': 'y'}],
    'layout': {'annotations': [{'font': {'size': 15}, 'showarrow': False, 'text': 'Training Data', 'x': 1000, 'y': 10},
                               {'font': {'size': 15}, 'showarrow': False, 'text': 'Test Data', 'x': 1300, 'y': 10}],
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'shapes': [{'line': {'color': 'green', 'dash': 'dash', 'width': 3},
                           'type': 'line',
                           'x0': 1164,
                           'x1': 1164,
                           'xref': 'x',
                           'y0': 0,
                           'y1': 1,
                           'yref': 'y domain'}],
               'template': '...',
               'title': {'text': 'Real vs Predicted Values for N-Beats'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'index'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

In [36]:
# initialize
nbeats = NBeatsModel(model_type = 'generic', lookback = 7, horizon = 1)
nbeats.build_layer()
# this is the base NBeats layer that’s used to construct the model
nbeats.model_layer

In [37]:
from tensorflow import keras

time_input = keras.layers.Input(shape = (7, ))
nbeats_layer = nbeats.model_layer(time_input)
non_time_input = keras.layers.Input(shape = (3,))
x = keras.layers.Dense(32, activation = 'relu')(non_time_input)
concat = keras.layers.Concatenate()([nbeats_layer, x])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs = [time_input, non_time_input], outputs = output)

In [38]:
# create training data from non time values
# excluding the first 7 values to match our time series data
X_non_time = df[['humidity', 'wind_speed', 'meanpressure']].values[7:]
# compile the model and fit
model.compile(loss = 'mae')
model.fit([X, X_non_time], y, verbose = False)

In [67]:
test = pd.DataFrame()
test['label'] = ['a'] * 10 + ['b'] * 10
test['value'] = [i for i in range(10)] * 2

In [69]:
test

,label,value
0,a,0
1,a,1
2,a,2
3,a,3
4,a,4
5,a,5
6,a,6
7,a,7
8,a,8
9,a,9


In [83]:
from kerasbeats import prep_multiple_time_series
windows, labels = prep_multiple_time_series(test, label_col = 'label', data_col = 'value', lookback = 7, horizon = 1)

In [84]:
windows

array([[0, 1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6, 7],
       [2, 3, 4, 5, 6, 7, 8],
       [0, 1, 2, 3, 4, 5, 6],
       [1, 2, 3, 4, 5, 6, 7],
       [2, 3, 4, 5, 6, 7, 8]])

In [82]:
labels

array([[6, 7],
       [7, 8],
       [8, 9],
       [6, 7],
       [7, 8],
       [8, 9]])